In [8]:
# Comparación entre imágenes filtradas por RED Y PROGRAMA DE FILTRADO
import numpy as np

def calcular_diferencia_porcentual_ajustada(array1_path, array2_path):
    """
    Calcula la diferencia porcentual entre dos arrays NumPy elemento por elemento,
    ajustando las dimensiones si difieren por una unidad en la última dimensión.

    Args:
        array1_path (str): Ruta al primer archivo .npy.
        array2_path (str): Ruta al segundo archivo .npy.

    Returns:
        numpy.ndarray: Un array con las diferencias porcentuales.
                        Retorna None si ocurre un error o las dimensiones no son ajustables.
    """
    try:
        arr1 = np.load(array1_path)
        arr2 = np.load(array2_path)
    except FileNotFoundError:
        print(f"Error: Asegúrate de que los archivos '{array1_path}' y '{array2_path}' existan en el directorio.")
        return None
    except Exception as e:
        print(f"Error al cargar los archivos .npy: {e}")
        return None

    # Verificar y ajustar las dimensiones
    if arr1.shape == arr2.shape:
        print("Las dimensiones de los arrays ya coinciden. No se requiere ajuste.")
    elif arr1.ndim != arr2.ndim:
        print(f"Error: Los arrays tienen un número diferente de dimensiones: {arr1.ndim} vs {arr2.ndim}.")
        return None
    else:
        # Encontrar el array más grande y el más pequeño
        if arr1.shape[:-1] == arr2.shape[:-1] and abs(arr1.shape[-1] - arr2.shape[-1]) == 1:
            if arr1.shape[-1] > arr2.shape[-1]:
                print(f"Ajustando '{array1_path}' de {arr1.shape} a {arr2.shape}...")
                arr1 = arr1[..., :arr2.shape[-1]] # Recortar la última dimensión de arr1
            else: # arr2.shape[-1] > arr1.shape[-1]
                print(f"Ajustando '{array2_path}' de {arr2.shape} a {arr1.shape}...")
                arr2 = arr2[..., :arr1.shape[-1]] # Recortar la última dimensión de arr2
        else:
            print(f"Error: Las dimensiones de los arrays no coinciden y no se pueden ajustar automáticamente de la forma esperada.")
            print(f"Dimensiones: {arr1.shape} vs {arr2.shape}")
            print("Este script solo ajusta la última dimensión si difieren por una unidad y el resto de dimensiones coinciden.")
            return None

    # Volver a verificar que las dimensiones ahora coinciden
    if arr1.shape != arr2.shape:
        print("Error: Las dimensiones de los arrays no coinciden después del intento de ajuste.")
        print(f"Dimensiones finales: {arr1.shape} vs {arr2.shape}")
        return None

    diferencia_porcentual = np.where(arr1 != 0, np.abs(((arr2 - arr1) / arr1) * 100), 0)

    return diferencia_porcentual

if __name__ == "__main__":
    ml_cube_path = "ml_cube.npy"
    filtered_cube_path = "filtered_cube.npy"

    print(f"Intentando calcular y ajustar la diferencia porcentual entre {ml_cube_path} y {filtered_cube_path}...")

    diferencias = calcular_diferencia_porcentual_ajustada(ml_cube_path, filtered_cube_path)

    if diferencias is not None:
        print("\nDiferencia porcentual (magnitud del cambio, filtered_cube vs ml_cube) después del ajuste:")

        print(diferencias.flatten()[:10], "...")

        print(f"\nValor mínimo de la diferencia porcentual: {np.min(diferencias):.2f}%")
        print(f"Valor máximo de la diferencia porcentual: {np.max(diferencias):.2f}%")


        max_diferencia = np.max(diferencias)

        count_max_pixels = np.sum(np.isclose(diferencias, max_diferencia))
        print(f"Cantidad de píxeles con la diferencia porcentual máxima ({max_diferencia:.2f}%): {count_max_pixels}")


        print(f"Valor promedio de la diferencia porcentual: {np.mean(diferencias):.2f}%")
        print(f"Desviación estándar de la diferencia porcentual: {np.std(diferencias):.2f}%")


Intentando calcular y ajustar la diferencia porcentual entre ml_cube.npy y filtered_cube.npy...
Ajustando 'ml_cube.npy' de (80, 1988, 1987) a (80, 1988, 1986)...

Diferencia porcentual (magnitud del cambio, filtered_cube vs ml_cube) después del ajuste:
[59.07134  53.212883 46.731396 46.808517 55.754387 52.759064 39.63813
 29.386288 20.61515  15.640247] ...

Valor mínimo de la diferencia porcentual: 0.00%
Valor máximo de la diferencia porcentual: 187.29%
Cantidad de píxeles con la diferencia porcentual máxima (187.29%): 1
Valor promedio de la diferencia porcentual: 2.28%
Desviación estándar de la diferencia porcentual: 1.77%
